# Text Classification with Cleanlab, TensorFlow, & SciKeras

This tutorial will use Cleanlab to find potential label errors in the IMDb movie review dataset. This dataset contains 50,000 labeled text reviews split evenly in the train and test set. Each review is labeled with a binary sentiment polarity label - positive (1) or negative (0). Cleanlab will shortlist *hundreds* of examples that confuses our ML model the most; many of which are potential label errors, edge cases and obscure examples.

**Overview of what we'll do in this tutorial:**

- Build a simple TensorFlow & Keras neural net and wrap it with SciKeras to make it scikit-learn compatible.

- Compute the out-of-sample predicted probabilities, ``pyx``, with cross validation.

- Generate a list of potential label errors with Cleanlab's ``get_noise_indices``.

- Build and train aa robust model with Cleanlab's ``LearningWithNoisyLabels`` wrapper. 

**Data:** https://ai.stanford.edu/~amaas/data/sentiment/

In [ ]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

## **1. Install the required dependencies**

``%%capture`` is a magic function to hide the cell's output.

In [ ]:
%%capture

%pip install cleanlab sklearn pandas tensorflow tensorflow-datasets scikeras

## **2. Load the ACL's IMDb dataset**

Load the full dataset from TensorFlow Dataset. 

In [ ]:
import tensorflow_datasets as tfds

raw_full_ds = tfds.load(name='imdb_reviews', split=('train+test'), batch_size=-1, as_supervised=True)

Split the dataset into two numpy arrays:
1. ``raw_full_texts`` for the movie reviews in text format, and
2. ``full_labels`` for the labels.

In [ ]:
raw_full_texts, full_labels = tfds.as_numpy(raw_full_ds)

## **3. Preprocess the text data**

Define a function that can standardize the text data in three steps:
1. Convert it to lower case
2. Remove the HTML break tags, ``<br />``
3. Remove any punctuation marks

In [ ]:
import tensorflow as tf
import re
import string

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html, f'[{re.escape(string.punctuation)}]','')

Create a ``TextVectorization`` layer that can standardize (by running the ``custom_standardization`` function we've just defined above), tokenize and vectorize our text data.

In [ ]:
from tensorflow.keras import layers

max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

Adapting ``vectorize_layer`` on our text data creates a mapping of each token to an integer. After that, we can vectorize our text data with the adapted ``vectorize_layer``. Finally, we'll also convert our text data into a numpy array as required by Cleanlab.

In [ ]:
vectorize_layer.adapt(raw_full_texts)

full_texts = vectorize_layer(raw_full_texts)

full_texts = full_texts.numpy()

## **4. Build a classifcation model**

Here, we build and compile a simple neural network with TensorFlow and Keras.

In [ ]:
from tensorflow.keras import losses, metrics

model = tf.keras.Sequential([
  tf.keras.Input(shape=(None,), dtype="int64"),
  layers.Embedding(max_features + 1, 16),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.compile(optimizer='adam',
              loss=losses.BinaryCrossentropy(from_logits=True),
              metrics=metrics.BinaryAccuracy())

## **5. Wrap with SciKeras for scikit-learn compatibility**

As some of Cleanlab's feature requires scikit-learn compatibility, we will need to adapt the above TensorFlow & Keras neural net accordingly. SciKeras is a convenient package that helps with this, read more about it here: https://www.adriangb.com/scikeras/stable/

In [ ]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(model, epochs=10)

## **6. Compute the out-of-sample predicted probabilities with cross validation**

We will fit the entire dataset on the model used to compute the out-of-sample predicted probabilities, ``pyx``, with cross validation. This model will not be used for model evaluation.

In [ ]:
_ = model.fit(full_texts, full_labels)

Compute the out-of-sample predicted probabilities, ``pyx``, with cross validation.

In [ ]:
%%capture

from sklearn.model_selection import cross_val_predict

pyx = cross_val_predict(model, full_texts, full_labels, cv=3, method='predict_proba')

## **7. Run Cleanlab's to find potential label errors**

Cleanlab has a ``get_noise_indices`` function to generate a list of potential label errors. Setting ``sorted_index_method="prob_given_label"`` returns the indices of all the most likely label errors, sorted by the most suspicious example first.

In [ ]:
from cleanlab.pruning import get_noise_indices

ordered_label_errors = get_noise_indices(
    s=full_labels,
    psx=pyx,
    sorted_index_method="prob_given_label"
)

## **8. Review some of the highest potential label errors**

In [ ]:
print(f"Cleanlab found {len(ordered_label_errors)} potential label errors. Here are the indices of the top 10 most likely ones: \n {ordered_label_errors[:10]}")

Change pandas display max column width to ``None`` and define a new function, ``print_as_df``, that can print any example from the raw dataset with just its index number.

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

def print_as_df(index):
    return pd.DataFrame({'texts': raw_full_texts[index], 'labels': full_labels[index]}, [index])

Review labeled as positive (1), but should be negative (0). Here are some review snippets:

> - "...incredibly **awful** score..."
>
> - "...**worst** Foley work ever done."
>
> - "...script is **incomprehensible**..."
>
> - "...editing is just **bizarre**."
>
> - "...**atrocious** pan and scan..."
>
> - "...**incoherent mess**..."
>
> - "...**amateur** directing there."

In [ ]:
print_as_df(44582)

Review labeled as positive (1), but should be negative (0). Here are some review snippets:

> - "...film seems **cheap**."
>
> - "...unbelievably **bad**..."
>
> - "...cinematography is **badly** lit..."
>
> - "...everything looking **grainy** and **ugly**."
>
> - "...sound is so **terrible**..."

In [ ]:
print_as_df(10404)

Review labeled as positive (1), but should be negative (0). Here are some review snippets:

> - "...hard to imagine a **boring** shark movie..."
>
> - "**Poor focus** in some scenes made the production seems **amateurish**."
>
> - "...**do nothing** to take advantage of..."
>
> - "...**far too few** scenes of any depth or variety."
>
> - "...just **look flat**...no contrast of depth..."
>
> - "...**introspective** and **dull**...constant **disappointment**."

In [ ]:
print_as_df(30151)

Cleanlab has shortlisted the most likely label errors to speed up your data cleaning process. With this list, you can decide whether to fix label errors, augment edge cases or remove obscure examples. 

These human-in-the-loop processes may be time-consuming, so if you'd like Cleanlab to automatically remove these noisy examples and train a model directly on the partially mislabeled dataset, you're in luck! Cleanlab provides a ``LearningWithNoisyLabels`` wrapper to do precisely this:

## **9. Adapt with Cleanlab's wrapper and train a robust model**

Similar to before, we will load the dataset, but this time, we will load the train and test set separately.

In [ ]:
raw_train_ds = tfds.load(name='imdb_reviews', split='train', batch_size=-1, as_supervised=True)
raw_test_ds = tfds.load(name='imdb_reviews', split='test', batch_size=-1, as_supervised=True)

raw_train_texts, train_labels = tfds.as_numpy(raw_train_ds)
raw_test_texts, test_labels = tfds.as_numpy(raw_test_ds)

We will use the same ``vectorize_layer`` as before, but we will reset its state and adapt it only on the train set. 

In [ ]:
vectorize_layer.reset_state() 

vectorize_layer.adapt(raw_train_texts) 

Vectorize the text data in the train and test sets, then convert them into numpy arrays.

In [ ]:
train_texts = vectorize_layer(raw_train_texts) 
test_texts = vectorize_layer(raw_test_texts)

train_texts = train_texts.numpy()
test_texts = test_texts.numpy()

We use the ``clone`` function to construct a new unfitted model then wrap it with Cleanlab's ``LearningWithNoisyLabels`` wrapper.

In [ ]:
from sklearn.base import clone
from cleanlab.classification import LearningWithNoisyLabels

model = clone(model)

lnl = LearningWithNoisyLabels(clf=model)

Train the wrapped model, ``lnl``, on the train set. 

In [ ]:
_ = lnl.fit(train_texts, train_labels)

## **10. Evaluate the robust model's performance**

In [ ]:
from sklearn.metrics import accuracy_score

pred_labels = lnl.predict(test_texts)
accuracy_score(test_labels, pred_labels)

## **What's next?**

Congratulation on completing this tutorial! Check out our following tutorial on using Cleanlab for tabular data classification!